### 1. Configure script.

In [1]:
import os, sys
GISFolder = os.getcwd()

In [2]:
# Note: gostnet.py and associated files are in the current working directory.

os.getcwd()

'C:\\Users\\grace\\GIS\\puerto rico'

In [3]:
import GOSTnet as gn # Python couldn't find the module. Moved it into C:\Users\grace\Anaconda3\envs\access\Lib\site-packages
import pandas as pd
from geopandas import GeoDataFrame
import shapely
from shapely.geometry import Point, box
import geopandas as gpd
import osmnx as ox
import networkx as nx
import numpy as np
import rasterio as rt

In [4]:
# Didn't use in this iteration:
import fiona
import peartree
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [5]:
pth = os.path.join(GISFolder, "data\working files\gn")
pth

'C:\\Users\\grace\\GIS\\puerto rico\\data\\working files\\gn'

### 3. Origins and destinations

Measure distance from origin/destination to nearest node and save to file.

In [6]:
gTime = nx.read_gpickle("gTime.pickle")

In [7]:
#%% If already created, load from file.
inOsnap = os.path.join(GISFolder, "data\working files\gn", "inOsnap.csv")
inOsnap = pd.read_csv(inOsnap)
inDsnap = os.path.join(GISFolder, "data\working files\gn", "inDsnap.csv")
inDsnap = pd.read_csv(inDsnap)
inHsnap = os.path.join(GISFolder, "data\working files\gn", "inHsnap.csv")
inHsnap = pd.read_csv(inHsnap)
inPsnap = os.path.join(GISFolder, "data\working files\gn", "inPsnap.csv")
inPsnap = pd.read_csv(inPsnap)

### Create travel time values for the road nodes nearest to each service.

Using calculate_OD.

In [8]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
origins = list(inOsnap.NN.unique())

In [9]:
listD = list(inDsnap.NN.unique()) 
listH = list(inHsnap.NN.unique()) 
listP = list(inPsnap.NN.unique()) 
destslist = listD + listH + listP
dests = list(set(destslist))

In [10]:
len(origins) # 142,421 unique nearest nodes.

142421

In [11]:
len(dests) # 1,011 unique nearest nodes.

1011

In [12]:
len(listH)

69

In [13]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [14]:
gn.example_edge(gTime)

(238026752, 238075156, {'osmid': 22137085, 'name': 'Calle F', 'highway': 'residential', 'oneway': False, 'length': 108.717, 'time': 19.569059999999997, 'mode': 'drive'})


In [15]:
OD = gn.calculate_OD(gTime, origins, dests, fail_value, weight = 'time')
# Took maybe 10 min.

In [16]:
OD_df = pd.DataFrame(OD, index = origins, columns = dests)

In [17]:
OD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142421 entries, 239148931 to 238880909
Columns: 1011 entries, 233324546 to 238196735
dtypes: float64(1011)
memory usage: 1.1 GB


In [18]:
OD_df.tail()

,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,238549012,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
238421388,16460.909360,9043.596294,15744.659018,5208.659860,2342.319710,9050.396574,4839.264443,5076.230410,10351.917856,9372.719094,...,2921.715720,15689.594294,4980.377940,3438.475700,5774.741691,5272.487050,4844.558670,9072.961134,5887.507214,7355.777344
238402093,16463.189540,9045.876474,15746.939198,5210.940040,2344.599890,9052.676754,4841.544623,5078.510590,10354.198036,9374.999274,...,2923.995900,15691.874474,4982.658120,3440.755880,5777.021871,5274.767230,4846.838850,9075.241314,5889.787394,7358.057524
238591938,15784.758289,9889.020974,14846.235913,6054.084540,1328.141310,9895.821254,5684.689123,5921.655090,7634.230394,10218.143774,...,3767.140400,14899.938121,5825.802620,4283.900380,6620.166371,6117.911730,5689.983350,9918.385814,6732.931894,8201.202024
238923898,12579.883760,10193.486526,11641.361384,6358.550092,5461.305768,10200.286806,5989.154675,6226.120642,4429.355866,10522.609326,...,5132.108802,11695.063592,6130.268172,4616.556562,6924.631923,6422.377282,7694.247972,10222.851366,7037.397446,8326.705734
238880909,12643.176800,10309.089546,11704.654424,6474.153112,5576.908788,10315.889826,6104.757695,6341.723662,4492.648906,10638.212346,...,5247.711822,11758.356632,6245.871192,4732.159582,7040.234943,6537.980302,7809.850992,10338.454386,7153.000466,8442.308754


In [19]:
# Convert to minutes and save to file.
OD_min = OD_df[OD_df <fail_value] / 60
OD_min.to_csv(os.path.join(pth, 'OD.csv'))
# Takes a couple minutes.

In [20]:
OD_min.tail(20)

,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,238549012,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
238061534,187.171666,63.549782,175.234161,28.914269,103.213949,63.663120,21.915226,15.546127,160.599907,69.035162,...,53.520838,174.316415,19.681828,44.106790,23.175239,21.669444,46.455046,64.039196,22.409153,37.968271
6242335058,203.884840,80.262956,191.947335,16.720849,102.226044,80.376294,24.895487,22.367670,159.612002,85.748336,...,52.532933,191.029589,15.279582,43.118885,4.118551,13.606323,29.057902,80.752370,38.378015,54.681446
238061543,187.203799,63.581915,175.266294,28.946402,103.246082,63.695253,21.947359,15.578260,160.632040,69.067295,...,53.552971,174.348548,19.713961,44.138923,23.207372,21.701577,46.487179,64.071329,22.441286,38.000404
5148259911,187.018202,63.396317,175.080696,28.061892,102.361571,63.509655,21.062848,14.693750,159.747529,68.881697,...,52.668461,174.162950,18.829451,43.254413,22.322862,20.817066,45.602669,63.885731,22.255688,37.814807
238061666,185.635126,62.013241,173.697620,26.967958,101.267637,62.126579,19.968914,13.599816,158.653595,67.498621,...,51.574527,172.779874,17.735517,42.160479,21.228928,19.723132,44.508735,62.502655,20.872612,36.431731
238061667,185.716719,62.094834,173.779213,27.048857,101.348536,62.208172,20.049813,13.680715,158.734494,67.580214,...,51.655426,172.861467,17.816416,42.241378,21.309827,19.804031,44.589634,62.584248,20.954205,36.513324
238065788,188.186183,64.564298,176.248677,27.790195,102.089874,64.677636,20.791151,14.422053,159.475832,70.049678,...,52.396764,175.330931,18.557754,42.982716,22.051165,20.545369,45.330972,65.053712,23.423669,38.982788
238349709,189.310829,65.688944,177.373323,27.159625,101.459304,65.802282,20.160581,13.791483,158.845262,71.174324,...,51.766194,176.455577,17.927184,42.352146,21.420595,19.914799,44.700402,66.178358,24.548315,40.107434
238347326,189.154319,65.532434,177.216813,26.913120,101.212799,65.645772,19.914076,13.544978,158.598757,71.017814,...,51.519689,176.299067,17.680679,42.105641,21.174090,19.668294,44.453897,66.021848,24.391805,39.950924
238261379,188.816837,65.194952,176.879331,26.575638,100.875317,65.308290,19.576594,13.207496,158.261275,70.680332,...,51.182207,175.961585,17.343197,41.768159,20.836608,19.330812,44.116415,65.684366,24.054323,39.613442


In [22]:
# Create POI-specific OD and save to file.
ODD = OD_df.loc[:, listD]
ODD = ODD[ODD < fail_value] / 60 
ODD.to_csv(os.path.join(pth, 'ODD.csv')) # Each takes a minute or so.

ODH = OD_df.loc[:, listH]
ODH = ODH[ODH < fail_value] / 60 
ODH.to_csv(os.path.join(pth, 'ODH.csv'))

ODP = OD_df.loc[:, listP]
ODP = ODP[ODP < fail_value] / 60 
ODP.to_csv(os.path.join(pth, 'ODP.csv'))

In [25]:
ODH.tail()

,238527834,243546298,238597596,237778206,237895435,2132540420,499948755,237982345,4156205993,237978749,...,2903017551,2705040978,238982313,238978858,238942305,2902931361,5213391119,4155890330,5136150615,239272483
238421388,241.972505,259.008976,268.274985,125.460415,186.867688,187.633549,84.589187,93.750471,94.873959,94.882669,...,82.178917,81.328726,84.189454,85.974378,80.446971,81.618336,224.353764,133.803914,73.341172,214.221954
238402093,242.010508,259.046979,268.312988,125.498418,186.905691,187.671552,84.627190,93.788474,94.911962,94.920672,...,82.216920,81.366729,84.227457,86.012381,80.484974,81.656339,224.391767,133.841917,73.379175,214.259957
238591938,196.677714,271.529266,269.485649,110.432711,200.958100,201.723961,36.901675,107.840882,108.964370,108.973081,...,96.269329,95.419138,98.279866,100.064789,94.537383,95.708748,191.230468,147.894326,90.835023,168.927163
238923898,143.263138,218.114691,216.071074,58.525861,175.604847,176.370708,40.303370,112.915308,114.038796,114.047506,...,101.343755,100.493563,103.354292,105.139215,99.611809,100.783174,137.815892,152.968752,166.654393,115.512588
238880909,144.318022,219.169575,217.125958,60.452578,176.659731,177.425592,42.230087,114.842025,115.965513,115.974223,...,103.270472,102.420280,105.281009,107.065932,101.538526,102.709891,138.870776,154.895469,168.581110,116.567472


### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [26]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
ODD = os.path.join(pth, "ODD.csv")
ODD = pd.read_csv(ODD)
ODP = os.path.join(pth, "ODP.csv")
ODP = pd.read_csv(ODP)
ODH = os.path.join(pth, "ODH.csv")
ODH = pd.read_csv(ODH)

In [35]:
ODH.tail(10)

,NN,238527834,243546298,238597596,237778206,237895435,2132540420,499948755,237982345,4156205993,...,2903017551,2705040978,238982313,238978858,238942305,2902931361,5213391119,4155890330,5136150615,239272483
142411,238547482,250.722303,234.125653,243.391662,149.668008,161.984366,162.750227,133.085593,71.998822,73.552705,...,58.624406,58.305095,57.173795,55.603295,57.676113,58.847477,199.470441,108.920592,57.615050,238.429547
142412,238537241,255.384843,238.788193,248.054202,154.330548,166.646906,167.412767,128.380875,76.661362,78.215245,...,63.286946,62.967634,61.836335,60.265835,62.338652,63.510018,204.132981,113.583132,52.910332,243.092087
142413,5138198157,255.425676,238.829026,248.095035,154.371381,166.687739,167.453600,128.062476,76.702195,78.256078,...,63.327779,63.008467,61.877168,60.306668,62.379485,63.550850,204.173814,113.623965,52.591933,243.132920
142414,238521463,255.701645,239.104995,248.371004,154.647350,166.963708,167.729569,127.119228,76.978164,78.532047,...,63.603748,63.284436,62.153137,60.582637,62.655454,63.826819,204.449784,113.899934,51.648685,243.408889
142415,238217078,279.565363,262.968713,272.234722,168.750242,190.827426,191.593287,127.443616,100.841882,102.395765,...,87.467466,87.148154,86.016855,84.446355,86.519172,87.690537,228.313502,137.763652,51.973073,257.511781
142416,238421388,241.972505,259.008976,268.274985,125.460415,186.867688,187.633549,84.589187,93.750471,94.873959,...,82.178917,81.328726,84.189454,85.974378,80.446971,81.618336,224.353764,133.803914,73.341172,214.221954
142417,238402093,242.010508,259.046979,268.312988,125.498418,186.905691,187.671552,84.627190,93.788474,94.911962,...,82.216920,81.366729,84.227457,86.012381,80.484974,81.656339,224.391767,133.841917,73.379175,214.259957
142418,238591938,196.677714,271.529266,269.485649,110.432711,200.958100,201.723961,36.901675,107.840882,108.964370,...,96.269329,95.419138,98.279866,100.064789,94.537383,95.708748,191.230468,147.894326,90.835023,168.927163
142419,238923898,143.263138,218.114691,216.071074,58.525861,175.604847,176.370708,40.303370,112.915308,114.038796,...,101.343755,100.493563,103.354292,105.139215,99.611809,100.783174,137.815892,152.968752,166.654393,115.512588
142420,238880909,144.318022,219.169575,217.125958,60.452578,176.659731,177.425592,42.230087,114.842025,115.965513,...,103.270472,102.420280,105.281009,107.065932,101.538526,102.709891,138.870776,154.895469,168.581110,116.567472


In [34]:
ODH.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) # Repeat for each OD set, if needed.

#### Find first nearest POI for each origin node. Run this block for each variable.

In [36]:
ODD["1D"] = 0
sub = ODD.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
ODD["1D"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
D1 = ODD[['NN', '1D']] # Remove unnecessary OD values.
D1.to_csv(os.path.join(pth, '1D_base.csv'))

In [37]:
D1.head(10) # Validation: The nodes that had NaN values in the OD matrix show a higher travel time in the landslide set. 

,NN,1D
0,239148931,37.489956
1,239077120,56.710245
2,239127767,54.502221
3,239136364,54.187776
4,239077188,54.715068
5,239176627,36.540948
6,237676072,39.801646
7,239149053,52.248120
8,237686017,58.995387
9,239158919,33.653093


In [38]:
ODH["1H"] = 0
sub = ODH.iloc[:,1:-1] # Filtering out the newly created field and the node ID column.
ODH["1H"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
H1 = ODH[['NN', '1H']] # Remove unnecessary OD values.
H1.to_csv(os.path.join(pth, '1H_base.csv'))

In [39]:
H1.head(10)

,NN,1H
0,239148931,39.231936
1,239077120,34.486956
2,239127767,32.278933
3,239136364,31.964488
4,239077188,32.491779
5,239176627,38.282928
6,237676072,28.394310
7,239149053,30.024832
8,237686017,36.772098
9,239158919,28.402349


In [40]:
ODP["1P"] = 0
sub = ODP.iloc[:,1:-1] # Filtering out the newly created field and the node ID column.
ODP["1P"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
P1 = ODP[['NN', '1P']] # Remove unnecessary OD values.
P1.to_csv(os.path.join(pth, '1P_base.csv'))

In [41]:
P1.head(10)

,NN,1P
0,239148931,14.262696
1,239077120,19.665247
2,239127767,17.457223
3,239136364,17.142778
4,239077188,17.670070
5,239176627,13.313688
6,237676072,28.394310
7,239149053,15.203122
8,237686017,21.950389
9,239158919,24.843397


#### Compare to baseline values.

### Create multi-modal travel times by combining walk time to road with drive time to nth nearest service.

In [42]:
# If starting new session, re-load from disk.
zwalk = os.path.join(pth, "zwalk.csv") 
zwalk = pd.read_csv(zwalk)

In [59]:
D1 = os.path.join(pth, "1D.csv")
D1 = pd.read_csv(D1)

In [43]:
zwalk.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183


In [44]:
D1.head()

,NN,1D
0,239148931,37.489956
1,239077120,56.710245
2,239127767,54.502221
3,239136364,54.187776
4,239077188,54.715068


In [45]:
# Merge nearest POIs and walktimes
zwalkD = zwalk.merge(D1, how='left', left_on='NN', right_on='NN', sort=False)
zwalkD.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1D
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,37.489956
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,37.489956
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,37.489956
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,37.489956
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,56.710245


In [46]:
# Merge nearest POIs and walktimes
zwalkP = zwalk.merge(P1, how='left', left_on='NN', right_on='NN', sort=False)
zwalkP.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1P
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,14.262696
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,14.262696
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,14.262696
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,14.262696
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,19.665247


In [47]:
# Merge nearest POIs and walktimes
zwalkH = zwalk.merge(H1, how='left', left_on='NN', right_on='NN', sort=False)
zwalkH.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1H
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,39.231936
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,39.231936
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,39.231936
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,39.231936
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,34.486956


In [48]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkH["mm1H"] = 0
zwalkH["mm1H"] = zwalkH["walk_time"] + zwalkH["1H"]

In [50]:
zwalkH.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1H,mm1H
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,39.231936,49.230541
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,39.231936,47.492890
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,39.231936,46.522424
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,39.231936,47.011877
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,34.486956,47.458140


In [51]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkP["mm1P"] = 0
zwalkP["mm1P"] = zwalkP["walk_time"] + zwalkP["1P"]

In [52]:
zwalkP.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1P,mm1P
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,14.262696,24.261301
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,14.262696,22.523650
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,14.262696,21.553184
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,14.262696,22.042637
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,19.665247,32.636430


In [53]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkD["mm1D"] = 0
zwalkD["mm1D"] = zwalkD["walk_time"] + zwalkD["1D"]

In [54]:
zwalkD.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1D,mm1D
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,37.489956,47.488561
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,37.489956,45.750910
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,37.489956,44.780444
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,37.489956,45.269897
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,56.710245,69.681428


In [55]:
zwalkD = zwalkD[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1D', 'mm1D']]
zwalkH = zwalkH[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1H', 'mm1H']]
zwalkP = zwalkP[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1P', 'mm1P']]

In [56]:
zwalkH.to_csv(os.path.join(pth, 'H_mm.csv'))
zwalkP.to_csv(os.path.join(pth, 'P_mm.csv'))
zwalkD.to_csv(os.path.join(pth, 'D_mm.csv'))